# MASH analysis pipeline with data-driven prior matrices

In this notebook, we utilize the MASH prior, referred to as the [mixture_prior](https://github.com/cumc/xqtl-pipeline/blob/6c637645ce16aee2aa7dc86bbc334fb6bb66b9d9/code/multivariate/MASH/mixture_prior.ipynb#L4), from a previous step. Our objective is to conduct a multivariate analysis under the MASH model. After fitting the model, we subsequently compute the posteriors for our variables of interest.

## Methods

### Multivariate adaptive shrinkage (MASH) analysis of eQTL data


Since we published Urbut 2019, we have improved implementation of MASH algorithm and made a new R package, [`mashr`](https://github.com/stephenslab/mashr). Major improvements compared to Urbut 2019 are:

1. Faster computation of likelihood and posterior quantities via matrix algebra tricks and a C++ implementation.
2. Faster computation of MASH mixture via convex optimization.
3. New ways to estimate prior in place of the `SFA` approach, see `mixture_prior.ipynb` for details.
4. Improve estimate of residual variance $\hat{V}$.

At this point, the input data have already been converted from the original association summary statistics to a format convenient for analysis in MASH.

## MWE Data

Avaiable on [synapse.org](https://www.synapse.org/#!Synapse:syn52624471)

## Multivariate analysis with MASH model

Using MWE with [prior previously generated](https://github.com/cumc/xqtl-pipeline/blob/main/code/multivariate/MASH/mixture_prior.ipynb).

In [5]:
sos run pipeline/mash_fit.ipynb mash \
    --output-prefix protocol_example \
    --data output/protocol_example.mashr_input.rds \
    --vhat-data output/mashr/protocol_example.ed_mixture.EZ.V_simple.rds \
    --prior-data output/mashr/protocol_example.ed_mixture.EZ.prior.rds \
    --compute-posterior \
    --cwd output/mashr \
    --container oras://ghcr.io/cumc/stephenslab_apptainer:latest

INFO: Running mash_1: Fit MASH mixture model (time estimate: <15min for 70K by 49 matrix)
INFO: mash_1 (index=0) is ignored due to saved signature
INFO: mash_1 output:   /home/gw/Documents/xQTL/output/mashr/protocol_example.EZ.mash_model.rds
INFO: Running mash_2: Compute posterior for the "strong" set of data as in Urbut et al 2017. This is optional because most of the time we want to apply the MASH model learned on much larger data-set.
INFO: mash_2 is completed.
INFO: mash_2 output:   /home/gw/Documents/xQTL/output/mashr/protocol_example.EZ.posterior.rds
INFO: Workflow mash (ID=wdb1a3918af0fd3c3) is executed successfully with 1 completed step and 1 ignored step.


In [10]:
readRDS('output/mashr/protocol_example.EZ.posterior.rds')$lfsr

ALL,Ast,End,Exc,Inh,Mic,OPC,Oli
1.000725e-06,1.206902e-01,0.38024501,2.341416e-05,0.021238026,2.438671e-02,2.822754e-01,4.950394e-02
6.195229e-01,6.725207e-01,0.65046008,6.835928e-01,0.582288619,6.287649e-01,6.903822e-01,4.929722e-01
5.502451e-08,6.952073e-05,0.42849153,3.848081e-09,0.001349139,2.974207e-01,2.267577e-01,2.193946e-01
1.141256e-02,3.673827e-01,0.29984283,8.920542e-03,0.062520281,1.557580e-01,1.438823e-01,3.652727e-01
4.803276e-01,1.793390e-01,0.01916176,1.752551e-01,0.384661176,3.138391e-01,4.888775e-01,3.793039e-01
3.240321e-01,1.006760e-04,0.28423516,2.692014e-10,0.464610846,8.359305e-04,8.654122e-21,1.358764e-09
4.744668e-01,3.330669e-16,0.35728521,2.052281e-05,0.437127648,3.887724e-02,1.843843e-05,1.374751e-06
3.166308e-03,0.000000e+00,0.34573821,2.111164e-04,0.424465824,1.686171e-02,9.992007e-16,2.119553e-07
4.826046e-04,3.121070e-02,0.31763916,3.181940e-01,0.000107790,6.340894e-19,2.832298e-02,3.138958e-02
2.426264e-01,3.950080e-01,0.16192905,1.023271e-01,0.284297524,3.167406e-04,1.449052e-02,4.348748e-01


## Global parameter settings

In [2]:
[global]
parameter: cwd = path('./mashr_workflow_output')
# Input summary statistics data
parameter: data = path
parameter: prior_data = path
parameter: vhat_data = path
# Prefix of output files. If not specified, it will derive it from data.
# If it is specified, for example, `--output-prefix AnalysisResults`
# It will save output files as `{cwd}/AnalysisResults*`.
parameter: output_prefix = ''
parameter: output_suffix = 'all'
# Exchangable effect (EE) or exchangable z-scores (EZ)
parameter: effect_model = 'EZ'
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
data = data.absolute()
cwd = cwd.absolute()
prior_data = prior_data.absolute()
vhat_data = vhat_data.absolute()
if len(output_prefix) == 0:
    output_prefix = f"{data:bn}"

mash_model = file_target(f"{cwd:a}/{output_prefix}.{effect_model}.mash_model.rds")

def sort_uniq(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

## `mashr` mixture model fitting


In [9]:
# Fit MASH mixture model (time estimate: <15min for 70K by 49 matrix)
[mash_1]
parameter: output_level = 4
input: data, vhat_data, prior_data
output: mash_model

task: trunk_workers = 1, walltime = '36h', trunk_size = 1, mem = '16G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout", container = container, entrypoint=entrypoint
    library(mashr)
    dat = readRDS(${_input[0]:r})
    vhat = readRDS(${_input[1]:r})
    U = readRDS(${_input[2]:r})$U
    mash_data = mash_set_data(dat$random.b, Shat=dat$random.s, alpha=${1 if effect_model == 'EZ' else 0}, V=vhat, zero_Bhat_Shat_reset = 1E3)
    saveRDS(list(mash_model = mash(mash_data, Ulist = U, outputlevel = ${output_level}), vhat_file=${_input[1]:r}, prior_file=${_input[2]:r}), ${_output:r})

### Optional posterior computations

Additionally provide posterior for the "strong" set in MASH input data.

In [10]:
# Compute posterior for the "strong" set of data as in Urbut et al 2017.
# This is optional because most of the time we want to apply the 
# MASH model learned on much larger data-set.
[mash_2]
# default to True; use --no-compute-posterior to disable this
parameter: compute_posterior = True
# input Vhat file for the batch of posterior data
skip_if(not compute_posterior)

input: data, vhat_data, mash_model
output: f"{cwd:a}/{output_prefix}.{effect_model}.posterior.rds"

task: trunk_workers = 1, walltime = '36h', trunk_size = 1, mem = '16G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout", container = container, entrypoint=entrypoint
    library(mashr)
    dat = readRDS(${_input[0]:r})
    vhat = readRDS(${_input[1]:r})
    mash_data = mash_set_data(dat$strong.b, Shat=dat$strong.s, alpha=${1 if effect_model == 'EZ' else 0}, V=vhat, zero_Bhat_Shat_reset = 1E3)
    mash_model = readRDS(${_input[2]:ar})$mash_model
    saveRDS(mash_compute_posterior_matrices(mash_model, mash_data), ${_output:r})